# Version Info
v5 version에서 cycleGAN 부분을 아래 notebook을 바탕으로 수정 중 == v6 version<br>https://colab.research.google.com/drive/1n5OzIaic1Ho60UAtqEgCnRyn7Lg1Po_v<br>sequence 길이를 모두 1로 맞춰야 함, 그렇지 않으면 생성할 때 마다 차원이 감소함. 아니면 train 과정에서 모델이 seq 길이 만큼 하나의 데이터에 대해 반복을 수행해서 생성을 진행해야 한다.

v6 version lstm cell return_sequnces=True로 설정하고 마지막 output layer에서 2차원 dense를 수행<br> --> seq_l을 데이터의 차원으로 갖도록 함

cycleGAN의 generation 부분 -> v7

cycleGAN의 loss 및 훈련 -> v8

gradients tape 문제 해결을 위한 tf dataset 이용 -> v9

v9의 cycleGAN pt type code 정리 및 파이프라인 보수 -> v10<br>
v10에서 seq화 과정에서 발생하는 데이터 손실, loss 그래프, gpu 사용량 최적화, notes to MIDI(생성 데이터 MIDI로 변환) 수정 -> v11<br>
v11에서 todo 6~ 까지의 수정 -> v12<br>
v12에서 vocab size 관련 error 수정 + batch 등의 파라미터 조정 -> v13<br>
v13에서 만든 pt type 완성본을 여러가지 방면으로 실험 -> v14<br>
v14에서 실험+ -> v15<br>
v15에서 seq, batch, bi, gru, tf.reduce_mean 실험 pt type(그나마 괜찮은 값) -> v16<br>
v16에서 심알 포스터에 들어갈 자료 생성 -> v17<br>
v17에서 발생하는 생성자의 학습 문제 및 과적합 문제 해결을 위해 판별자 lr 낮추기 -> 판별자의 학습을 늦춰 생성자의 학습 시간을 벌어주기, 
binary crossentropy 쪽에서 발생 또는 다른 loss 계산에서 발생하는 nan or inf 문제를 해결하기 위해 loss function 수정 -> v18<br>
v18 + TimeDistributed model 적용한 거 -> v19<br>

# TODO
1. 일반 LSTM model과는 다르게 cycleGAN의 input data를 만들어야 한다.
즉, sequence data로 data를 변환할 때, 일반 LSTM model과는 달리 target data를 생성하지 않아야 하며 seq_L 만큼 마다 나눠서 데이터 손실 없이 input data로 사용해야한다.

    -> 모델이 seq_l 길이 만큼의 데이터를 바탕으로 (그 길이 안 데이터 해석) output을 만들어 낸다.<br> 이후 sequence data로 나뉘어진 output data를 다시 reshape한 후, midi로 변환하면 된다.

2. sequence data 및 음악적 특성을 충분히 고려할 수 있도록 discriminator 및 그에 따른 loss function 구현 필요 -- 시간 계획 필수 (기한 고려)

3. 향후 cycleGAN과 관련된 loss(cycle consistnecy 등)도 손을 볼 수 있으면 좋을 것 같다.

4. GPU 사용량 늘리기

5. batch를 만들때, shuffle 주의하기, test data에는 사용하지 않아야 할수도 있다. -> train data shuffle을 수행해도 무방할 정도의 seq_l을 잡거나 shuffle을 수행하지 않아야 한다.

6. reshape 이용해서 create sequence 부분 고치기

7. 실제 연구 데이터를 사용할 때, 두 도메인의 데이터 길이 조심하기 - 학습과정에서

8. notes_to_midi 잘 이용하기

9. MIDI file 제대로, 전부 append 되는 지 확인하기 --> file_path glob 부분에서 slicing에 의한 문제 --> 해결

10. cycleGAN 생성 과정 중에 pitch, step, duration의 loss 차이가 튼 영향을 미친다면 music_generation 노트북을 활용하여 loss 재정의하기

11. 각 노래 midi 별로 나눠서 처리하거나, batch size 크기를 늘려 모델이 batch로 학습을 진행할 때, 각각의 노래가 섞이는 경우를 조금은 줄이는 것이 좋은지 실험 필요

12. colab 환경에서 여러개의 notebook을 띄워 놓은 후, 각각의 변수들을 변경하면서 실험 진행<br>

13. 12를 진행하는 과정에서 실험 리스트, 설계 과정, 결과 등을 모두 기록해야함

14. colab notebook 최대 가용량 -> 실험 가능량 설계 -> 시간 계획



# Experiment plan
1. batch
2. data<br>
    1. count
    2. type<br>
        1. musician<br>
            1. piano-e-competition
        2. elec<br>
            1. maestro
3. generator model<br>
    1. model structure<br>
        1. type<br>
            1. bidirectional
            2. Dense
            3. GRU
        2. layer<br>
            1. unit count
    2. hyper parameter
4. discriminator model
    1. model structure<br>
        1. type<br>
            1. bidirectional
            2. Dense
            3. GRU
        2. layer<br>
            1. unit count
    2. hyper parameter<br>
    3. for music<br>
        1. patchGAN
5. loss function weight<br>
    1. brute force method
6. optimizer<br>
    1. type
    2. learning rate
7. future plan<br>
    1. 특정곡의 곡 또는 작곡관련 nlp 정보 -> embbding -> 새로운 층으로 concatenate -> multimodal<br>
    2. 

In [ ]:
# # for installing

# !pip install -U -q pip
# !pip install -q tensorboardX
# !pip install -q git+https://github.com/tensorflow/examples.git
# !pip install -q pretty_midi
# !pip install -q knockknock

In [ ]:
# !git clone https://github.com/chelate1118/piano-e-competition-midi-files.git

fatal: destination path 'piano-e-competition-midi-files' already exists and is not an empty directory.


In [ ]:
# !git clone https://github.com/bytedance/GiantMIDI-Piano.git

Cloning into 'GiantMIDI-Piano'...
Updating files:  97% (287/294)
Updating files:  98% (289/294)
Updating files:  99% (292/294)
Updating files: 100% (294/294)
Updating files: 100% (294/294), done.


In [ ]:
import sys
print(sys.executable)

c:\Users\gobak\.virtualenvs\.venv-HOEqjjdC\Scripts\python.exe


In [ ]:
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="9"

In [ ]:
# import 과정 error -> AVX2를 이용한 tf compile 진행 시, 더 최적화 가능함을 알려줌 | 아래 링크 참조
# http://erpcomputing.com/build-tensorflow-from-source--mkl-enabled

#library import

# for data and preprocessing
import tensorflow_datasets as tfds
import collections
import glob
import numpy as np
import pathlib
import pandas as pd
import pretty_midi
import seaborn as sns
from sklearn.model_selection import train_test_split
# import MIDI_Pedal # 사용법 : MIDI_Pedal.apply('test.mid', 'output.mid')


# for model
# 설치가 git인 경우 local에서 설치한 후 .venv/Lib에 넣으면 됨, local pip 설치경로는 위에 있음
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, GRU, LeakyReLU, Activation, TimeDistributed
from keras.layers import TimeDistributed, GRU, Dense, Dropout, Conv2D, BatchNormalization, MaxPool2D, GlobalMaxPool2D
import keras 

# for the others
# import fluidsynth # MIDI는 다른 거로 재생하기
import datetime
from IPython import display
from IPython.display import clear_output
import matplotlib.pyplot as plt
from typing import Dict, List, Optional, Sequence, Tuple
import time
import os
# from knockknock import desktop_sender
# from google.colab import files # google colab에서 결과 midi file 다운받고 싶을 때 import 하면 됨.

In [ ]:
# archive_data_dir = pathlib.Path('archive')
# data_l = len(glob.glob(str(archive_data_dir/'**/*.mid*')))
# print(data_l)


295


In [ ]:
# giant midi piano dataset

giantmidi_data_dir = pathlib.Path('giant_midis')
giantmidi_filenames = glob.glob(str(giantmidi_data_dir/'*.mid*'))
print('Number of files :', len(giantmidi_filenames))

giantmidi_sample_file = giantmidi_filenames[0]
print(giantmidi_sample_file)

Number of files : 10854
giant_midis\A., Jag, Je t'aime Juliette, OXC7Fd0ZN8o.mid


In [ ]:
# maestro dataset

maestro_data_dir = pathlib.Path('data/maestro-v2.0.0')
if not maestro_data_dir.exists():
  tf.keras.utils.get_file(
      'maestro-v2.0.0-midi.zip',
      origin='https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip',
      extract=True,
      cache_dir='.', cache_subdir='data',
  )

maestro_filenames = glob.glob(str(maestro_data_dir/'**/*.mid*'))
print('Number of files :', len(maestro_filenames))

maestro_sample_file = maestro_filenames[0]
print(maestro_sample_file)

Number of files : 1282
data\maestro-v2.0.0\2004\MIDI-Unprocessed_XP_15_R2_2004_01_ORIG_MID--AUDIO_15_R2_2004_03_Track03_wav.midi


In [ ]:
pm = pretty_midi.PrettyMIDI(maestro_sample_file)
pm_test = pretty_midi.PrettyMIDI(giantmidi_sample_file)

instrument = pm.instruments[0]
instrument_test = pm_test.instruments[0]
instrument_name = pretty_midi.program_to_instrument_name(instrument.program)
instrument_name_test = pretty_midi.program_to_instrument_name(instrument_test.program)

if not instrument_name == instrument_name_test:
    raise Exception('both dataset has incompatiable instrument')

print('Number of instruments:', len(pm.instruments))
print('Instrument name:', instrument_name)

Number of instruments: 1
Instrument name: Acoustic Grand Piano


In [ ]:
# global variable declaration

#global
seed = 42

# for model
AUTOTUNE = tf.data.AUTOTUNE
tf.random.set_seed(seed)
BUFFER_SIZE = -1
BATCH_SIZE = 1 # cycleGAN tf document 참고한 값 
SEQ_LENGTH = 1024
VOCAB_SIZE = 128

N_FEATURE = OUTPUT_CHANNELS = 3
LR_gen = 2e-4 # opt
LR_disc = 2e-4
# B1 = 0.05 # opt

# for data
np.random.seed(seed)
# MUSICIAN_MIDI_FILES_DIR = pathlib.Path('data/maestro-v2.0.0')
MUSICIAN_MIDI_FILES_DIR = pathlib.Path('data/maestro-v2.0.0')
ELEC_MIDI_FILES_DIR = pathlib.Path('giant_midis')
DATA_NUM = 1000

# for the others
# Sampling rate for audio playback
_SAMPLING_RATE = 16000
PRINT_ITER = 50

용어 정리<br>real : 실제 연주자 연주<br>elec : 전자 음악 연주

In [ ]:
def midi_to_notes(midi_file: str) -> pd.DataFrame:
  pm = pretty_midi.PrettyMIDI(midi_file)
  instrument = pm.instruments[0]
  notes = collections.defaultdict(list)

  # Sort the notes by start time
  sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
  prev_start = sorted_notes[0].start

  for note in sorted_notes:
    start = note.start
    end = note.end
    notes['pitch'].append(note.pitch)
    notes['start'].append(start)
    notes['end'].append(end)
    notes['step'].append(start - prev_start)
    notes['duration'].append(end - start)
    prev_start = start

  return pd.DataFrame({name: (np.array(value) if name == 'pitch' else np.float32(np.array(value))) for name, value in notes.items()})

def plot_piano_roll(notes: pd.DataFrame, count: Optional[int] = None):
    if count:
      title = f'First {count} notes'
    else:
      title = f'Whole track'
      count = len(notes['pitch'])
    plt.figure(figsize=(20, 4))
    plot_pitch = np.stack([notes['pitch'], notes['pitch']], axis=0)
    plot_start_stop = np.stack([notes['start'], notes['end']], axis=0)
    plt.plot(
        plot_start_stop[:, :count], plot_pitch[:, :count], color="b", marker=".")
    plt.xlabel('Time [s]')
    plt.ylabel('Pitch')
    _ = plt.title(title)

def plot_distributions(notes: pd.DataFrame, drop_percentile=2.5):
  plt.figure(figsize=[15, 5])
  plt.subplot(1, 3, 1)
  sns.histplot(notes, x="pitch", bins=20)

  plt.subplot(1, 3, 2)
  max_step = np.percentile(notes['step'], 100 - drop_percentile)
  sns.histplot(notes, x="step", bins=np.linspace(0, max_step, 21))
  
  plt.subplot(1, 3, 3)
  max_duration = np.percentile(notes['duration'], 100 - drop_percentile)
  sns.histplot(notes, x="duration", bins=np.linspace(0, max_duration, 21))

def notes_to_midi(
  notes: pd.DataFrame,
  out_file: str, 
  instrument_name: str,
  velocity: int = 100,  # note loudness
) -> pretty_midi.PrettyMIDI:

  pm = pretty_midi.PrettyMIDI()
  instrument = pretty_midi.Instrument(
      program=pretty_midi.instrument_name_to_program(
          instrument_name))

  prev_start = 0
  for i, note in notes.iterrows():
    start = float(prev_start + note['step'])
    end = float(start + note['duration'])
    note = pretty_midi.Note(
        velocity=velocity,
        pitch=int(note['pitch']),
        start=start,
        end=end,
    )
    instrument.notes.append(note)
    prev_start = start

  pm.instruments.append(instrument)
  pm.write(out_file)
  return pm

##### 참고
<span style='font-size:80%'>1. midi_to_notes(midi_file: str) -> pd.DataFrame<br>2. notes_to_midi(
  notes: pd.DataFrame,
  out_file: str, 
  instrument_name: str,
  velocity: int = 100,  # note loudness
) -> pretty_midi.PrettyMIDI<br>3. plot_piano_roll(notes: pd.DataFrame, count: Optional[int] = None)<br>4. def plot_distributions(notes: pd.DataFrame, drop_percentile=2.5)</span>

In [ ]:
# DATA_NUM = 10

In [ ]:
# data import musician AND FAKE
# error 많이 나면 np.array 이용하기
# filenames slicing 수 적절히 설정하기

# musician_filenames = glob.glob(str(MUSICIAN_MIDI_FILES_DIR/'**/*.mid*'))
musician_filenames = glob.glob(str(MUSICIAN_MIDI_FILES_DIR/'**/*.mid*'))
# musician => 2700개 정도 있음
musician_filenames = musician_filenames[:DATA_NUM]

elec_filenames = glob.glob(str(ELEC_MIDI_FILES_DIR/'*.mid*'))
elec_filenames = elec_filenames[:DATA_NUM]

print(len(musician_filenames))
print(len(elec_filenames))


musician_all_notes = []
elec_all_notes = []

# 각 notes listz
# 0 : musician - 사람이 연주한 데이터
# 1 : fake - 사람이 연주하지 않은 데이터
data_notes = [musician_all_notes, elec_all_notes]
data_paths = [musician_filenames, elec_filenames]
data_notes_ds = {} # tfds를 이용한 slices 이후 데이터임, (n_notes, tf_dt) tuple형의 데이터가 원소인 list

train_data_notes_ds = [] # n_notes, dataset(slice)의 tuple 형태
validation_data_notes_ds = []
test_data_notes_ds = []

data_split_ratio = {'train' : 0.8, 'validation' : 0.2, 'test' : 0.2}

key_order = ['pitch', 'step', 'duration']
not_key_order = ['start', 'end']

data_type = ['musician', 'elec']

# notes 수를 맞추기 위함
n_notes = 10e9

file_cnt = 0

print(f'{len(musician_filenames) + len(elec_filenames)} files are ready to process')
print('-' * PRINT_ITER)
print()
print('Start!')
print()
time.sleep(2)

start_time = time.time()

for i, data_path in enumerate(data_paths):
    for f in data_path:
        notes = midi_to_notes(f)
        data_notes[i].append(notes)
        if file_cnt % 10 == 0:
            clear_output(wait=True)
            print(f'process {(file_cnt / (len(musician_filenames) + len(elec_filenames))) * 100.0:.3f}% {file_cnt}th file process continue...')
        file_cnt += 1
        
    # append와 concat은 다르다!
    data_notes[i] = pd.concat(data_notes[i])
    data_notes[i].drop(columns=not_key_order, inplace=True)
    n_notes = min(len(data_notes[i]), n_notes)    
    
for i, data_path in enumerate(data_paths):
    _train, _test = data_notes[i][:int(n_notes * (1 - data_split_ratio['test']))], data_notes[i][int(n_notes * (1 - data_split_ratio['test'])):]
    _train, _validation = _train[:int(len(_train) * data_split_ratio['train'])], _train[int(len(_train) * data_split_ratio['train']):]
    n__train = len(_train)
    n__validation = len(_validation)
    n__test = len(_test)
    data_dict = {'train' : (n__train, _train), 'validation' : (n__validation, _validation), 'test' : (n__test, _test)}
    data_notes_ds[data_type[i]] = data_dict
    
    
print()
print('End!')

print(f'total process time for {len(musician_filenames) + len(elec_filenames)} files : {time.time() - start_time:.3f}sec')
# data_notes_ds Info
# (n_data, tfds) 원소,
# musician ==> train, validation, test
# elec ==> train, validation, test
# 총 6개의 원소가 들어가 있음

process 50.000% 10th file process continue...

End!
total process time for 20 files : 3.434sec


In [ ]:
data_notes_ds['musician']['train'][1].isnull().any()

pitch       False
step        False
duration    False
dtype: bool

In [ ]:
data_notes_ds['musician']['train'][1]['step']

0       0.000000
1       0.186458
2       0.009375
3       0.175000
4       0.169792
          ...   
8072    0.003125
8073    0.011458
8074    0.014583
8075    0.015625
8076    0.197917
Name: step, Length: 26371, dtype: float32

In [ ]:
data_notes_ds['musician']['train'][1]

,pitch,step,duration
0,71,0.000000,0.096875
1,55,0.186458,0.217708
2,71,0.009375,0.505208
3,59,0.175000,0.167708
4,62,0.169792,0.119792
...,...,...,...
8072,72,0.003125,0.042708
8073,74,0.011458,0.072917
8074,69,0.014583,0.037500
8075,57,0.015625,0.038542


In [ ]:
data_notes_ds['elec']['train'][1]

,pitch,step,duration
0,45,0.000000,0.506510
1,84,0.001302,0.505208
2,48,0.009115,0.006510
3,76,0.000000,0.295573
4,52,0.143229,0.513021
...,...,...,...
4108,66,0.052083,0.023438
4109,93,0.179688,0.036458
4110,78,0.022135,0.029948
4111,74,0.000000,0.063802


In [ ]:
# input data nan or inf 확인
# min, max 값 확인해서 이상치 확인하기

#TODO
# pitch도 Dtype float64로 바꿔보기 / 아니면 전부 float32로 굳이 64까지 안 써도 무방
print(data_notes_ds['elec']['train'][1].info())
print(data_notes_ds['musician']['train'][1].info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26371 entries, 0 to 4112
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pitch     26371 non-null  int32  
 1   step      26371 non-null  float32
 2   duration  26371 non-null  float32
dtypes: float32(2), int32(1)
memory usage: 515.1 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 26371 entries, 0 to 8076
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pitch     26371 non-null  int32  
 1   step      26371 non-null  float32
 2   duration  26371 non-null  float32
dtypes: float32(2), int32(1)
memory usage: 515.1 KB
None


In [ ]:
print(data_notes_ds['elec']['train'][1].describe())
print(data_notes_ds['musician']['train'][1].describe())

              pitch          step      duration
count  26371.000000  26371.000000  26371.000000
mean      64.831444      0.143615      0.540759
std       12.745089      0.413116      0.644795
min       24.000000      0.000000      0.003906
25%       56.000000      0.002604      0.136719
50%       64.000000      0.016927      0.333333
75%       74.000000      0.165365      0.670573
max      103.000000     15.378906      6.005208
              pitch          step      duration
count  26371.000000  26371.000000  26371.000000
mean      64.845323      0.086609      0.165801
std       12.251832      0.170984      0.307159
min       24.000000      0.000000      0.001042
25%       55.000000      0.007292      0.051042
50%       66.000000      0.036458      0.086458
75%       74.000000      0.126042      0.156250
max      105.000000      8.341666      6.515625


# cycleGAN

# many to many에 맞게 gru 사용하기, timedistributed and reshape와 같은 함수 사용하기

In [ ]:
# v2
def get_generator(model_name, output_channels=OUTPUT_CHANNELS):
  input_shape = (SEQ_LENGTH, output_channels)
  # print(input_shape)
  
  # LSTM, Bi를 아래와 같이 단순히 쌓아도 되는지 불분명
  # onhot encoding이나 embedding처럼 차원 조절 방법 생각
  
  inputs = tf.keras.Input(input_shape, name='gen_input')
  x0 = Bidirectional(LSTM(3, return_sequences=True))(inputs)
  x1 = Bidirectional(LSTM(6, return_sequences=True))(x0)
  
  x1 = Bidirectional(LSTM(12, return_sequences=True))(x1)
  x1 = Bidirectional(LSTM(12, return_sequences=True))(x1)
  
  x1 = Bidirectional(LSTM(24, return_sequences=True))(x1)
  x1 = Bidirectional(LSTM(24, return_sequences=True))(x1)
  
  x1 = Bidirectional(LSTM(12, return_sequences=True))(x1)
  x1 = Bidirectional(LSTM(12, return_sequences=True))(x1)

  x1 = Bidirectional(LSTM(6, return_sequences=True))(x1)
  x2 = Bidirectional(LSTM(3, return_sequences=True))(x1)
  
  outputs = TimeDistributed(Dense(output_channels), name='gen_output')(x2)
  # 마지막 layer는 timedistributed layer로! 다른 거도 가능하면 고려하기
  model = tf.keras.Model(inputs, outputs, name=model_name)
  return model


# 향후 수정 후, 사용하기
def get_discriminator(model_name, output_channels=OUTPUT_CHANNELS):
  input_shape = (SEQ_LENGTH, output_channels)
  # print(input_shape)
  inputs = tf.keras.Input(input_shape, name='disc_input')
  x0 = Bidirectional(LSTM(3, return_sequences=True))(inputs)
  
  x1 = Bidirectional(LSTM(6, return_sequences=True))(x0)
  x1 = Bidirectional(LSTM(12, return_sequences=True))(x1)
  x1 = Bidirectional(LSTM(12, return_sequences=True))(x1)
  x1 = Bidirectional(LSTM(12, return_sequences=True))(x1)

  x2 = Bidirectional(LSTM(12, return_sequences=True))(x1)
  
  # activation='sigmoid'를 loss nan 문제 해결방법으로 사용해보기
  outputs = TimeDistributed(Dense(24), name='disc_output')(x2)
  
  # 마지막 layer는 timedistributed layer로! 다른 거도 가능하면 고려하기
  model = tf.keras.Model(inputs, outputs, name=model_name)
  return model
    

# 향후 종류를 다양하게 변형하기 / 여기에서 pretrained model 이용함
# gen_musician = tf.keras.models.load_model('model_save/v17-2/gen_elec-epoch2.h5')
# gen_elec = tf.keras.models.load_model('model_save/v17-2/gen_elec-epoch2.h5')
gen_musician = get_generator('gen_musician')
gen_elec = get_generator('gen_elec')

disc_musician = get_discriminator('disc_musician')
disc_elec = get_discriminator('disc_elec')

In [ ]:
def build_convnet(shape=(112, 112, 3)):
    momentum = .9
    model = keras.Sequential()
    model.add(Conv2D(64, (3,3), input_shape=shape,
        padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization(momentum=momentum))
    
    # flatten...
    model.add(GlobalMaxPool2D())
    return model

In [ ]:
b = build_convnet()
b.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 112, 112, 64)      1792      
                                                                 
 conv2d_17 (Conv2D)          (None, 112, 112, 64)      36928     
                                                                 
 batch_normalization_8 (Batc  (None, 112, 112, 64)     256       
 hNormalization)                                                 
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                             
                                                                 
 conv2d_18 (Conv2D)          (None, 56, 56, 128)       73856     
                                                                 
 conv2d_19 (Conv2D)          (None, 56, 56, 128)      

In [ ]:
def action_model(shape=(5, 112, 112, 3), nbout=3):
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_convnet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (5, 112, 112, 3) shape
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(64))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(nbout, activation='softmax'))
    return model

In [ ]:
a = action_model()

In [ ]:
a.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_1 (TimeDis  (None, 5, 512)           4689216   
 tributed)                                                       
                                                                 
 gru_1 (GRU)                 (None, 64)                110976    
                                                                 
 dense_13 (Dense)            (None, 1024)              66560     
                                                                 
 dropout_3 (Dropout)         (None, 1024)              0         
                                                                 
 dense_14 (Dense)            (None, 512)               524800    
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                      

In [ ]:
gen_musician.summary()

Model: "gen_musician"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gen_input (InputLayer)      [(None, 1024, 3)]         0         
                                                                 
 bidirectional_32 (Bidirecti  (None, 1024, 6)          168       
 onal)                                                           
                                                                 
 bidirectional_33 (Bidirecti  (None, 1024, 12)         624       
 onal)                                                           
                                                                 
 bidirectional_34 (Bidirecti  (None, 1024, 24)         2400      
 onal)                                                           
                                                                 
 bidirectional_35 (Bidirecti  (None, 1024, 24)         3552      
 onal)                                                

In [ ]:
disc_musician.summary()

Model: "disc_musician"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 disc_input (InputLayer)     [(None, 1024, 3)]         0         
                                                                 
 bidirectional_52 (Bidirecti  (None, 1024, 6)          168       
 onal)                                                           
                                                                 
 bidirectional_53 (Bidirecti  (None, 1024, 12)         624       
 onal)                                                           
                                                                 
 bidirectional_54 (Bidirecti  (None, 1024, 24)         2400      
 onal)                                                           
                                                                 
 bidirectional_55 (Bidirecti  (None, 1024, 24)         3552      
 onal)                                               

In [ ]:
# loss function

loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)
# losses_ratio = {'disc_loss' : 1.0, 'gen_loss' : 1.0, 'cycle_loss' : 1.0}

for_nan_checking = {}
for_prev_nan_checking = {}

# first_one_checked = False
nan_checked = False

def disc_loss(real, generated):
#     real += tf.convert_to_tensor(1)
#     real = tf.math.scalar_mul(0.5, real)
    
#     generated += tf.convert_to_tensor(1)
#     generated = tf.math.scalar_mul(0.5, real)
    global for_nan_checking, for_prev_nan_checking, nan_checked
    
    real_loss = loss_obj(tf.ones_like(real), real)
    if real_loss != real_loss:
        for_nan_checking['real_loss'] = real
        nan_checked = True
    
    if not nan_checked:
        for_prev_nan_checking['real_loss'] = real
        
    
    fake_loss = loss_obj(tf.zeros_like(generated), generated)
    if fake_loss != fake_loss:
        for_nan_checking['fake_loss'] = generated
        nan_checked = True
        
    if not nan_checked:
        for_prev_nan_checking['fake_loss'] = generated
    
    total_loss = real_loss + fake_loss
    
    
    return total_loss * 0.5

def gen_loss(generated):
#     generated += tf.convert_to_tensor(1)
#     generated = tf.math.scalar_mul(0.5, generated)
    global for_nan_checking, for_prev_nan_checking, nan_checked

    t_gen_loss = loss_obj(tf.ones_like(generated), generated)
    
    if t_gen_loss != t_gen_loss:
        for_nan_checking['gen_loss'] = generated
        nan_checked = True
        
    if not nan_checked:
        for_prev_nan_checking['gen_loss'] = generated
        
    return t_gen_loss * 1

def cycle_loss(real_data, cycled_data):
    global for_nan_checking, for_prev_nan_checking, nan_checked
    
    t_cycle_loss = tf.reduce_mean(tf.abs(real_data - cycled_data))
    
    if t_cycle_loss != t_cycle_loss:
        for_nan_checking['cycle_loss'] = (real_data, cycled_data)
        nan_checked = True
        
    if not nan_checked:
        for_prev_nan_checking['cycle_loss'] = (real_data, cycled_data)
        
    return t_cycle_loss * 10

def iden_loss(real_data, gen_from_real_data):
    global for_nan_checking, for_prev_nan_checking, nan_checked
    
    t_iden_loss = tf.reduce_mean(tf.abs(real_data - gen_from_real_data))
    
    if t_iden_loss != t_iden_loss:
        for_nan_checking['iden_loss'] = (real_data, gen_from_real_data)
        nan_checked = True
        
    if not nan_checked:
        for_prev_nan_checking['iden_loss'] = (real_data, gen_from_real_data)
    
    return t_iden_loss * 5

In [ ]:
# optimizer

# LR_gen = 9e-4
# LR_disc = 1e-6
# B1 = 0.5 

gen_musician_opt = tf.keras.optimizers.Adam(learning_rate=LR_gen)
gen_elec_opt = tf.keras.optimizers.Adam(learning_rate=LR_gen)

disc_musician_opt = tf.keras.optimizers.Adam(learning_rate=LR_disc)
disc_elec_opt = tf.keras.optimizers.Adam(learning_rate=LR_disc)

In [ ]:
# # checkpoint_path의 경우 계속해서 바꿔줘야 에러가 나지 않음, 이 cell(+위에 model 정의 cell)을 반복해서 실행하면 error 발생함
# checkpoint_path = "./checkpoint/train_2022_11_10"

# ckpt = tf.train.Checkpoint(gen_musician=gen_musician,
#                            gen_elec=gen_elec,
#                            disc_musician=disc_musician,
#                            disc_elec=disc_elec,
#                            gen_musician_opt=gen_musician_opt,
#                            gen_elec_opt=gen_elec_opt,
#                            disc_musician_opt=disc_musician_opt,
#                            disc_elec_opt=disc_elec_opt)

# ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# # if a checkpoint exists, restore the latest checkpoint.
# if ckpt_manager.latest_checkpoint:
#   ckpt.restore(ckpt_manager.latest_checkpoint)
#   print ('Latest checkpoint restored!!')

In [ ]:
# real_x : musician data
# real_y : elec data
# g : gen elec
# f : gen music

@tf.function
def train_step(real_x, real_y, losses_ratio):
    # persistent is set to True because the tape is used more than
    # once to calculate the gradients.
    all_loss = []
    with tf.GradientTape(persistent=True) as tape:
        # Generator G translates X -> Y
        # Generator F translates Y -> X.
        fake_y = gen_elec(real_x, training=True)
        cycled_x = gen_musician(fake_y, training=True)

        fake_x = gen_musician(real_y, training=True)
        cycled_y = gen_elec(fake_x, training=True)

        # same_x and same_y are used for identity loss.
        same_x = gen_musician(real_x, training=True)
        same_y = gen_elec(real_y, training=True)
        
        real_x_pitch = np.round(real_x.numpy()[:, :, 0] * 128)
        fake_y_pitch = np.round(fake_y.numpy()[:, :, 0] * 128)
        
        real_y_pitch = np.round(real_y.numpy()[:, :, 0] * 128)
        fake_x_pitch = np.round(fake_x.numpy()[:, :, 0] * 128)
        
        # print(real_x_pitch[0], fake_y_pitch[0])
        
#         print(type((real_x_pitch == fake_y_pitch)))
        false_pitch_sum = np.sum(np.abs(real_x_pitch - fake_y_pitch)) + np.sum(np.abs(real_y_pitch - fake_x_pitch))
        
        false_pitch_sum = np.sqrt(false_pitch_sum).item()
        
        # 모든 pitch가 같아, false_pitch_sum == 0인 경우 방지 
        false_pitch_sum = false_pitch_sum if false_pitch_sum > 1.0 else 1
        
#         print(false_pitch_sum)
        
        
#         print(real_x_pitch)
        
        # print(real_x_pitch == fake_y_pitch)
        
        
#         print(real_x[0].numpy()[:, 0] == same_x[0].numpy()[:, 0])
#         print(type(same_x))
#         print(same_x.numpy())
        

        disc_real_x = disc_musician(real_x, training=True)
        disc_real_y = disc_elec(real_y, training=True)

        disc_fake_x = disc_musician(fake_x, training=True)
        disc_fake_y = disc_elec(fake_y, training=True)

        # calculate the loss
        gen_elec_loss = gen_loss(disc_fake_y) #* false_pitch_sum
        gen_musician_loss = gen_loss(disc_fake_x) #* false_pitch_sum
        total_cycle_loss = cycle_loss(real_x, cycled_x) + cycle_loss(real_y, cycled_y)

        elec_iden_loss = iden_loss(real_y, same_y)
        musician_iden_loss = iden_loss(real_x, same_x)

        # Total generator loss = adversarial loss + cycle loss
        total_gen_elec_loss = gen_elec_loss*losses_ratio['gen'] + total_cycle_loss*losses_ratio['cyc'] + elec_iden_loss*losses_ratio['iden']
        total_gen_musician_loss = gen_musician_loss*losses_ratio['gen'] + total_cycle_loss*losses_ratio['cyc'] + musician_iden_loss*losses_ratio['iden']

        disc_musician_loss = disc_loss(disc_real_x, disc_fake_x)*losses_ratio['disc']
        disc_elec_loss = disc_loss(disc_real_y, disc_fake_y)*losses_ratio['disc']

        all_loss = [tf.identity(gen_elec_loss), 
                    tf.identity(gen_musician_loss), 
                    tf.identity(total_cycle_loss), 
                    tf.identity(total_gen_elec_loss),
                    tf.identity(total_gen_musician_loss),
                    tf.identity(disc_musician_loss),
                    tf.identity(disc_elec_loss),
                    tf.identity(musician_iden_loss),
                    tf.identity(elec_iden_loss)]


    # Calculate the gradients for generator and discriminator
    gen_elec_gradients = tape.gradient(total_gen_elec_loss, 
                                          gen_elec.trainable_variables)
    
    gen_musician_gradients = tape.gradient(total_gen_musician_loss, 
                                          gen_musician.trainable_variables)
    
    disc_musician_gradients = tape.gradient(disc_musician_loss, 
                                              disc_musician.trainable_variables)
    disc_elec_gradients = tape.gradient(disc_elec_loss, 
                                              disc_elec.trainable_variables)

    # Apply the gradients to the optimizer
    gen_elec_opt.apply_gradients(zip(gen_elec_gradients, 
                                              gen_elec.trainable_variables))

    gen_musician_opt.apply_gradients(zip(gen_musician_gradients, 
                                              gen_musician.trainable_variables))
    
    disc_musician_opt.apply_gradients(zip(disc_musician_gradients,
                                                  disc_musician.trainable_variables))
    
    disc_elec_opt.apply_gradients(zip(disc_elec_gradients,
                                                  disc_elec.trainable_variables))
    
    return all_loss
    

In [ ]:
# data = np.array([[x + i*3 for x in range(3)] for i in range(1024)])
# data.shape

In [ ]:
def to_seq_data(data, seq_l):
    # data = np.array([[x + i*3 for x in range(3)] for i in range(1024)])
    seq_data_list = []
    
    for i in range(data.shape[0] - seq_l + 1):
        seq_data_list.append(np.expand_dims(data[i:i+seq_l], axis=0))
    
    seq_data = np.concatenate(seq_data_list, axis=0)
    
    return seq_data

In [ ]:
# a = to_seq_data(data=data, seq_l = 10)
# print(a.shape)
# print(a)

In [ ]:
def notes_to_trainable_dataset(all_notes, n_data):
    # 나중에 없애도 무방
    
    key_order = ['pitch', 'step', 'duration']
    train_notes = np.stack([all_notes[key] for key in key_order], axis=1)
    n_notes, n_features = train_notes.shape 

    train_notes = train_notes[:n_notes - (n_notes % SEQ_LENGTH)]
    train_notes[:, 0] = train_notes[:, 0] / VOCAB_SIZE

    seq_train_notes = to_seq_data(data=train_notes, seq_l=SEQ_LENGTH)
    # seq_train_notes = train_notes.reshape(-1, SEQ_LENGTH, 3)
    # seq_train_notes = seq_train_notes / np.array([VOCAB_SIZE, 0.0, 0.0])

    print(f'seq_train_notes shape {seq_train_notes.shape}')
    print()

    # 여기를 다르게 바꿈 / pretrain에서 사용한 data를 사용하지 않기 위해서 :n_data -> n_data:로
    seq_train_notes = seq_train_notes[:n_data]
    
    seq_ds = tf.data.Dataset.from_tensor_slices(seq_train_notes)
    print(f'notes_ds element spec {seq_ds.element_spec}')
    print()

    # BUFFER_SIZE = n_notes - SEQ_LENGTH
    train_ds = seq_ds.map(lambda x: (tf.cast(x, tf.float32)), num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(BATCH_SIZE, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
    print(f'train_ds spec {train_ds.element_spec}')
    print()

    print('-' * PRINT_ITER)

    return train_ds


musician_train_ds = notes_to_trainable_dataset(data_notes_ds['musician']['train'][1], 300)

elec_train_ds = notes_to_trainable_dataset(data_notes_ds['elec']['train'][1], 300)

seq_train_notes shape (24577, 1024, 3)

notes_ds element spec TensorSpec(shape=(1024, 3), dtype=tf.float64, name=None)

train_ds spec TensorSpec(shape=(None, 1024, 3), dtype=tf.float32, name=None)

--------------------------------------------------
seq_train_notes shape (24577, 1024, 3)

notes_ds element spec TensorSpec(shape=(1024, 3), dtype=tf.float64, name=None)

train_ds spec TensorSpec(shape=(None, 1024, 3), dtype=tf.float32, name=None)

--------------------------------------------------


In [ ]:
# @tf.function을 이용하여 code를 실행하는 도중에 .nmupy()와 같은 code 실행 가능
# https://github.com/tgjeon/TF-Eager-Execution-Guide-KR/blob/master/guide.md 참고
# https://www.tensorflow.org/api_docs/python/tf/config/run_functions_eagerly
tf.config.run_functions_eagerly(True)

In [ ]:
EPOCHS = 36
VERSION = '19'
SAVE_PATH = 'runs_test5'

In [ ]:
from tensorboardX import SummaryWriter
writer = SummaryWriter(f'{SAVE_PATH}/v{VERSION}-epoch{EPOCHS}')

# problem
1. nan 값 -> https://data-newbie.tistory.com/281


In [ ]:
update_cnt = 0

1. learning rate가 커서 gradient discent 과정에서 튕겨져 나감
2. learning rate의 크기 차이로 gen과 disc의 학습에 충분한 시간이 부여되지 않음

-> learning rate에 대해서 gen, disc 각각 작게, 크게 했을때 변화하는 값 찾기

In [ ]:
# tensorboard graph color rgb : #1fddff
# disc -> 틀렸을때, 강한 피드백(scalar)을 작용 -> 학습에 오랜 시간
losses_ratio = {'disc' : 1.0, 'gen' : 1.0, 'cyc' : 1.0, 'iden' : 1.0} # cycle, iden loss ratio는 서로 종속적임(in tf document)
all_loss = []
all_loss_name = ['gen_elec_loss', 
                 'gen_musician_loss',
                 'total_cycle_loss', 
                 'total_gen_elec_loss',
                 'total_gen_musician_loss',
                 'disc_musician_loss',
                 'disc_elec_loss',
                 'musician_iden_loss',
                 'elec_iden_loss']

%load_ext tensorboard
%tensorboard --logdir {SAVE_PATH} --host 0.0.0.0 --port=6011

# 단순히 모델의 가중치 업데이트 횟수를 의미함
# 향후 이 값을 바탕으로 epoch를 계산 가능 | 
# update_n = epoch * (n_notes / (seq_length * batch_size)) = epoch * (n_seq_notes / batch_size)

# update_cnt = 0

total_start_time = time.time()

for epoch in range(EPOCHS):
    start = time.time()
    n = 0
    
    for musician, elec in tf.data.Dataset.zip((musician_train_ds, elec_train_ds)):
        # musician = tf.cast(musician, dtype=tf.float32)
        # elec = tf.cast(elec, dtype=tf.float32)
                
        all_loss = [0 for _ in range(len(all_loss_name))]   

        for_all_loss = train_step(musician, elec, losses_ratio)
        for i, loss in enumerate(for_all_loss):
            all_loss[i] += loss.numpy().item()

        if update_cnt % 100 == 0:
            print('-', end='')
#         if update_cnt % 4500 == 0 and update_cnt != 0:
#             gen_musician.save(f'model_save/v{VERSION}/gen_musician-epoch{EPOCHS}-step4500.h5')
#             gen_elec.save(f'model_save/v{VERSION}/gen_elec-epoch{EPOCHS}-step4500.h5')
#             disc_musician.save(f'model_save/v{VERSION}/disc_musician-epoch{EPOCHS}-step4500.h5')
#             disc_elec.save(f'model_save/v{VERSION}/disc_elec-epoch{EPOCHS}.h5')
        
        n += 1
        update_cnt += 1

        # 아래 부분에서 BATCH_SIZE로 나눌 지에 관해 고민 필요
        all_loss = [loss for loss in all_loss]
        for loss, loss_name in zip(all_loss, all_loss_name):
            writer.add_scalars('check_info/', {loss_name : loss}, update_cnt)

    # clear_output(wait=True)
    
    # note를 시각화할 수 있는 거 넣기
    # ex) generate_images(generator_g, sample_horse)
    
#     if (epoch + 1) % 3 == 0:
#         ckpt_save_path = ckpt_manager.save()
#         print(f'Saving checkpoint for each {epoch + 1} at {ckpt_save_path}')
    
    print(f'Time taken for epoch {epoch + 1} is {time.time() - start:.3f} sec\n')

print()
print('-' * PRINT_ITER)
print(f'Time taken for total {EPOCHS} epoch : {time.time() - total_start_time:.3f} sec')
writer.close()

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6011 (pid 2023924), started 0:00:39 ago. (Use '!kill 2023924' to kill it.)

-

In [ ]:
def generate_domain_notes(input_ds, model):
    result_list = []
    generated_notes_list = []
    n_batch_data = tf.data.experimental.cardinality(input_ds).numpy().item()
    # n_batch_data = 1

    for inp in input_ds.take(n_batch_data):
        gen_music = model(inp)
        result_list.append(gen_music)

    for result in result_list:
        pitchs = result[:, :, 0]
        steps = result[:, :, 1]
        durations = result[:, :, 2]

        pitchs = tf.reshape(pitchs, [-1])
        pitchs = tf.math.scalar_mul(VOCAB_SIZE, pitchs)
        pitchs = tf.math.round(pitchs)
        pitchs = tf.cast(pitchs, dtype=tf.int32)
        pitchs = pitchs.numpy()

        durations = tf.reshape(durations, [-1])
        durations = durations.numpy()

        steps = tf.reshape(steps, [-1])
        steps = steps.numpy() 

        generated_notes = []
        prev_start = 0

        for pitch, duration, step in zip(pitchs, durations, steps):
            pitch, duration, step = pitch.item(), duration.item(), step.item()

            if pitch > VOCAB_SIZE - 1:
                pitch = VOCAB_SIZE - 1
            if pitch < 0:
                pitch = 0
                
            start = prev_start + step 
            end = start + duration
            input_note = (pitch, step, duration)
            generated_notes.append((*input_note, start, end))

            prev_start = start

        generated_notes = pd.DataFrame(
            generated_notes, columns=(*key_order, 'start', 'end')
        )

        generated_notes_list.append(generated_notes)

    generated_notes_all = pd.concat(generated_notes_list, ignore_index=True)
    return generated_notes_all

def notes_to_testable_dataset(all_notes):
    key_order = ['pitch', 'step', 'duration']
    test_notes = np.stack([all_notes[key] for key in key_order], axis=1)
    n_notes, n_features = test_notes.shape 
    
    n_notes_for_seq = n_notes - (n_notes % SEQ_LENGTH)
    test_notes = test_notes[:n_notes_for_seq]
    
    test_notes = test_notes.reshape(-1, SEQ_LENGTH, 3)
    print(f'test_notes shape {test_notes.shape}')
    print()
    
    notes_ds = tf.data.Dataset.from_tensor_slices(test_notes)
    print(f'notes_ds element spec {notes_ds.element_spec}')
    print()

    seq_ds = notes_ds
    print(f'seq_ds element_spec {seq_ds.element_spec}')
    print()

    # BUFFER_SIZE = n_notes - SEQ_LENGTH
    test_ds = seq_ds.batch(n_notes_for_seq, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(tf.data.experimental.AUTOTUNE)
    print(f'test_ds spec {test_ds.element_spec}')
    print()

    print('-' * PRINT_ITER)

    return test_ds

def generate_notes(midi_filepath, model):
    notes = midi_to_notes(midi_filepath)
    test_ds = notes_to_testable_dataset(notes)
    generated_notes = generate_domain_notes(test_ds, model)
    return generated_notes
    

midi_filepath = elec_filenames[100]

generated_notes = generate_notes(midi_filepath, gen_musician)
print(generated_notes)

In [ ]:
midi_filepath = elec_filenames[666]

generated_notes = generate_notes(midi_filepath, gen_musician)
generated_notes

In [ ]:
real_notes = midi_to_notes(midi_filepath)
real_notes

In [ ]:
# np.sqrt(np.sum(np.abs((generated_notes['pitch'] - real_notes['pitch'][len(generated_notes)]).to_numpy())) / (len(generated_notes) / (SEQ_LENGTH*BATCH_SIZE)))

In [ ]:
test_file = f'model_save/v{VERSION}/test.mid'
out_pm = notes_to_midi(
    generated_notes, out_file=test_file, instrument_name=instrument_name)

In [ ]:
out_file = f'model_save/v{VERSION}/output_gen_musician-epoch{EPOCHS}.mid'
out_pm = notes_to_midi(
    generated_notes, out_file=out_file, instrument_name=instrument_name)

In [ ]:
out_file = f'model_save/v{VERSION}/output_real_elec-epoch{EPOCHS}.mid'
out_pm = notes_to_midi(
    real_notes, out_file=out_file, instrument_name=instrument_name)

In [ ]:
gen_musician.save(f'model_save/v{VERSION}/gen_musician-epoch{EPOCHS}.h5')
gen_elec.save(f'model_save/v{VERSION}/gen_elec-epoch{EPOCHS}.h5')
disc_musician.save(f'model_save/v{VERSION}/disc_musician-epoch{EPOCHS}.h5')
disc_elec.save(f'model_save/v{VERSION}/disc_elec-epoch{EPOCHS}.h5')

In [ ]:
# # generator 불러오기
# gen_musician_ckpt_path = ''
# gen_musician.load_weights(gen_musician_ckpt_path)

# gen_elec_ckpt_path = '' 
# gen_elec.load_weights(gen_elec_ckpt_path)